LCEL for RAG
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝 试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在LangChainHub上找一个可用的RAG提示词模板，测试对比两者的召回率和生成质量。

In [7]:
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

In [11]:
file_path = "tsla-20231231-gen.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [42]:
print(len(docs))
print(docs[0].page_content[0:300])
print(docs[0].metadata)

130
UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year	ended	
December	31
,	2023
OR
o
TRANSITION	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	
{'source': 'tsla-20231231-gen.pdf', 'page': 0}


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [13]:
llm = ChatOpenAI(model="gpt-4o-mini")
prompt = hub.pull("oogijo/financial-advisor")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])

In [15]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
for chunk in rag_chain.stream("请给出特斯拉2023财务数据摘要"):
    print(chunk, end="", flush=True)

特斯拉2023财务数据摘要显示，公司的总收入为96,773百万美元，汽车销售收入为78,509百万美元，能源生成和存储收入为6,035百万美元。净收入为14,974百万美元，综合收入为15,215百万美元。总资产达到106,618百万美元。

In [17]:
for chunk in rag_chain.stream("请简要分析特斯拉2023年财务报告。"):
    print(chunk, end="", flush=True)

特斯拉2023年的财务报告显示，公司的总收入达到96,773百万美元，比2022年的81,462百万美元显著增长，主要得益于汽车销售和能源业务的增长。尽管总收入增加，公司的毛利润下降至17,660百万美元，显示出成本压力。净收入为14,974百万美元，相比2022年的12,587百万美元有所上升，表明公司仍保持盈利增长。

In [39]:
for chunk in rag_chain.stream("特斯拉2023盈利能力主要表现在哪些方面，将来可能存在的风险和主要竞争对手有哪些？"):
    print(chunk, end="", flush=True)

特斯拉2023年的盈利能力主要依赖于电动汽车的消费需求和高销量的产品销售。未来可能面临的风险包括竞争激烈的劳动市场、材料供应的波动，以及消费者对电动汽车需求的变化。主要竞争对手包括其他汽车制造商进入电动和替代燃料车辆市场的公司，尤其是在中国和欧洲的市场。

LCELMulti-chain(可选)
1. 实现一个多链版本的代码生成，输入功能需求，输出2种(Python，Java)以上编程
语言的代码实现

In [2]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


In [25]:
python_coder = (
    ChatPromptTemplate.from_template(
        "用Python语言实现{question}"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
java_coder = (
    ChatPromptTemplate.from_template(
        "用Java语言实现{question}"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
combiner = (
    ChatPromptTemplate.from_messages(
        [
            ("human", "分别用Python和Java编程语言实现同一个问题"),
            ("system", "Python实现:\n{python_codes}\n\nJava实现:\n{java_codes}")
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
super_coder = (
    {"question": RunnablePassthrough()}
    | {
        "java_codes": java_coder,
        "python_codes": python_coder,
    }
    |combiner
)

In [26]:
super_coder

{
  question: RunnablePassthrough(),
  java_codes: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='用Java语言实现{question}'))])
              | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10fc586e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fc10e90>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')
              | StrOutputParser(),
  python_codes: ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='用Python语言实现{question}'))])
                | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10fc49220>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fc4acc0>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_prox

In [27]:

result = super_coder.invoke({"question": "快速排序"})

In [28]:
print(result)

下面是用Python和Java实现快速排序的代码示例，已按照您的要求给出。

### Python实现

快速排序是一种高效的排序算法，采用分治法策略。以下是用Python实现快速排序的代码示例：

```python
def quick_sort(arr):
    # 如果数组长度小于等于1，直接返回
    if len(arr) <= 1:
        return arr
    else:
        # 选择一个基准值，这里选择数组的最后一个元素
        pivot = arr[-1]
        # 划分出小于基准值和大于基准值的子数组
        left = [x for x in arr[:-1] if x <= pivot]
        right = [x for x in arr[:-1] if x > pivot]
        # 递归排序左边和右边，然后合并结果
        return quick_sort(left) + [pivot] + quick_sort(right)

# 示例使用
if __name__ == "__main__":
    arr = [3, 6, 8, 10, 1, 2, 1]
    sorted_arr = quick_sort(arr)
    print("排序后的数组:", sorted_arr)
```

### Java实现

以下是用Java语言实现的快速排序算法：

```java
public class QuickSort {

    // 主方法，用于测试快速排序
    public static void main(String[] args) {
        int[] array = {10, 7, 8, 9, 1, 5};
        System.out.println("原始数组: ");
        printArray(array);
        
        quickSort(array, 0, array.length - 1);
        
        System.out.println("排序后的数组: ");
        printArray(array);
    }

